# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
""" Importing all required Python packages """ 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
"""Creating the list of all the event_data files names that will be consumed"""
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
"""Read all the event data .csv files and write the data into one single file event_datafile_new.csv, the data from this step will be loaded into the tables."""
full_data_rows_list = [] 
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
            
        for line in csvreader:
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
"""Print the total number of records in event_datafile_new.csv file"""
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [10]:
"""Create a new Apache Cassandra connection"""
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [11]:
"""Creating a new Keyspace called sparkifykeyspace, under which the tables will be created."""
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifykeyspace 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)
else:
    print("sparkifykeyspace Keyspace created successfully")

#### Set Keyspace

In [12]:
"""Set keyspace to sparkifykeyspace"""
try:
    session.set_keyspace('sparkifykeyspace')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [20]:
"""Create a new table called song_details with columns session_id, item_in_session, artist, song_title and song_length with primary key as session_id, item_in_session"""
create_song_details = "CREATE TABLE IF NOT EXISTS song_details"
create_song_details = create_song_details + (""" (session_id INT, item_in_session INT, artist TEXT, song_title TEXT,
                                        song_length FLOAT, PRIMARY KEY (session_id, item_in_session)
                                   )""")
try:
    session.execute(create_song_details)
except Exception as e:
    print(e)
else:
    print("song_details table created successfully!")
                        

song_details table created successfully!


In [21]:
"""Load the data from event_datafile_new.csv into song_details table"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:

        query = "INSERT INTO song_details(session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [22]:
"""Perform select on song_details table and print the results"""

select_song_details = """ SELECT artist, song_title, song_length FROM song_details
                    WHERE session_id = %s 
                    AND item_in_session = %s 
                 """
try:
    rows = session.execute(select_song_details, (338, 4))
except Exception as e:
    print(e)
else:
    print("song_details select result: ")
    
for row in rows:
    print(row.artist,row.song_title, str(row.song_length))

song_details select result: 
Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [23]:
"""Create a new table called artist_details with columns user_id, session_id, item_in_session, artist_name, song_tile, user_first_name, user_last_name and primary key columns user_id, 
    session_id and item_in_session"""
create_artist_details = "CREATE TABLE IF NOT EXISTS artist_details"
create_artist_details = create_artist_details + (""" (user_id INT, session_id INT, item_in_session INT, 
                                                        artist_name TEXT, song_title TEXT, user_first_name TEXT, 
                                                        user_last_name TEXT, PRIMARY KEY (user_id, session_id, item_in_session)
                                                )""")
try:
    session.execute(create_artist_details)
except Exception as e:
    print(e)
else:
    print("artist_details table created successfully")
              

artist_details table created successfully


In [24]:
"""Load the data from event_datafile_new.csv into artist_details table"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_details(user_id, session_id, item_in_session, artist_name, \
                                            song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [25]:
"""Perform select on artist_details table and print the results"""

select_artist_details = """ SELECT artist_name, song_title, user_first_name, user_last_name FROM artist_details
                            WHERE user_id = %s
                            AND session_id = %s 
                        """      
try:
    rows = session.execute(select_artist_details, (10, 182))
except Exception as e:
    print(e)
else:
    print("artist_details select result: ")
    
for row in rows:
    print(row.artist_name, row.song_title, row.user_first_name, row.user_last_name)

artist_details select result: 
Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [27]:
"""Create a new table called user_details with columns song_title, user_id, user_first_name and user_last_name and primary key columns song_title, user_id"""

create_user_details = "CREATE TABLE IF NOT EXISTS user_details"
create_user_details = create_user_details + (""" (song_title TEXT, user_id INT, user_first_name TEXT, user_last_name TEXT, 
                                                    PRIMARY KEY ((song_title), user_id)
                                            )""")
try:
    session.execute(create_user_details)
except Exception as e:
    print(e)
else:
    print("user_details table created successfully")
                    

user_details table created successfully


In [28]:
"""Load the data from event_datafile_new.csv into suser_details table"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_details(song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"

        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [29]:
"""Perform select on user_details table and print the results"""

select_user_details = """SELECT user_first_name, user_last_name FROM user_details WHERE song_title = %s;"""
print(select_user_details)

try:
    rows = session.execute(select_user_details, ('All Hands Against His Own', ))
except Exception as e:
    print(e)
else:
    print("user_details select result: ")

for row in rows:
    print(row.user_first_name, row.user_last_name)

 SELECT user_first_name, user_last_name FROM user_details WHERE song_title = %s;
user_details select result: 
Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
session.execute("DROP TABLE IF EXISTS song_details")
session.execute("DROP TABLE IF EXISTS artist_details")
session.execute("DROP TABLE IF EXISTS user_details")

### Close the session and cluster connection¶

In [9]:
session.shutdown()
cluster.shutdown()